In [0]:
%run "/Workspace/formual1/includes/configuration"

In [0]:
%run "/Workspace/formual1/includes/common_functions"

In [0]:
dbutils.widgets.help()

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source=dbutils.widgets.get("p_data_source")

In [0]:
v_data_source

In [0]:
    display(dbutils.fs.ls('/'))

In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DoubleType

In [0]:
circuits_schema=StructType(fields=[StructField("circuitId",IntegerType(),False),
                                   StructField("circuitRef",StringType(),True),
                                   StructField("name",StringType(),True),
                                   StructField("location",StringType(),True),
                                   StructField("country",StringType(),True),
                                   StructField("lat",DoubleType(),True),
                                   StructField("lng",DoubleType(),True),
                                   StructField("alt",IntegerType(),True),
                                   StructField("url",StringType(),True)
                                      ])

In [0]:

circuits_schema=spark.read.csv(f"{raw_folder_path}/circuits.csv",header=True,inferSchema=True)

In [0]:
display(circuits_schema)

Select required columns

In [0]:
circuits_schema_df=circuits_schema.select("circuitId","circuitRef","name","location","country","lat","lng","alt")

In [0]:
circuits_schema_df=circuits_schema.select(circuits_schema.circuitId,circuits_schema.circuitRef,circuits_schema.name,circuits_schema.location,circuits_schema.country,circuits_schema.lat,circuits_schema.lat,circuits_schema.lng,circuits_schema.alt)

In [0]:
circuits_schema_df=circuits_schema.select(circuits_schema["circuitId"],circuits_schema["circuitRef"],circuits_schema["name"],circuits_schema["location"],circuits_schema["country"],circuits_schema["lat"],circuits_schema["lng"],circuits_schema["alt"])

In [0]:
from pyspark.sql.functions import col
circuits_schema_df=circuits_schema.select(col("circuitId"),col("circuitRef"),col("name"),col("location"),col("country"),col("lat"),col("lng"),col("alt"))

In [0]:
display(circuits_schema_df)

Renaming columns

In [0]:
from pyspark.sql.functions import current_timestamp,lit
circuits_renamed_df=circuits_schema_df.withColumnRenamed("circuitId","circuit_id").withColumnRenamed("circuitRef","circuit_ref").withColumnRenamed("name","circuit_name").withColumnRenamed("location","circuit_location").withColumnRenamed("country","circuit_country").withColumnRenamed("lat","circuit_lat").withColumnRenamed("lng","circuit_lng").withColumnRenamed("alt","circuit_alt").withColumn("data_source",lit(v_data_source))
display(circuits_renamed_df)

In [0]:
## add ingestion date using function
circuits_final_df=add_ingestion_time(circuits_renamed_df)
display(circuits_final_df)

In [0]:
# from pyspark.sql.functions import current_timestamp,lit
# circuits_final_df=circuits_renamed_df.withColumn("ingestion_date",current_timestamp())\
# .withColumn("env",lit("Production"))
# display(circuits_final_df)


Write to parquet file


In [0]:
circuits_final_df=circuits_final_df.drop("env")

In [0]:
display(circuits_final_df)

In [0]:
circuits_final_df.write.parquet(f"{processed_folder_path}/circuits",mode="overwrite")

In [0]:
%fs
ls /mnt/storageformula1training/processed/circuits

In [0]:
df=spark.read.parquet("/mnt/storageformula1training/processed/circuits")
display(df)

In [0]:
dbutils.notebook.exit("Success")